In [20]:
! pip install pysrt

     |████████████████████████████████| 112kB 2.9MB/s eta 0:00:01
  Created wheel for pysrt: filename=pysrt-1.1.2-cp37-none-any.whl size=13426 sha256=3a449fd6aac02422ef68228a8329a5441c6ea5969182eeef220f2818f35b024a
  Stored in directory: /home/ismail/.cache/pip/wheels/57/7d/68/0307a3018abbaa1a7b870e6419123d07480dd169bfdea74aea
Successfully built pysrt


In [51]:
import os
import pysrt
import time
import pandas as pd

In [22]:
subtitles_path = 'data/urheiluruutu/'

In [72]:
def to_seconds(str_time):
    str_time = str_time.replace(',', '.')
    if '.' not in str_time: 
        str_time = str_time + '.000'
    assert(time.strptime(str_time,'%H:%M:%S.%f'))
    seconds = sum(x * float(t) for x, t in zip([3600, 60, 1], str_time.split(":")))
    return seconds

In [73]:
data = []
for filename in os.listdir(subtitles_path):
    print(subtitles_path + filename)
    program_id = filename.split(".mp4_fi.sr")[0]
    subs =  pysrt.open(subtitles_path + filename)
    for l in subs:
        sub = {'program': program_id}
        sub['content'] = str(l.text)
        sub['start'] = str(l.start)
        sub['start_s'] = to_seconds(str(l.start))
        sub['end'] = str(l.end)
        sub['end_s'] = to_seconds(str(l.end))
        sub['duration_s'] = sub['end_s'] - sub['start_s']
        data.append(sub)

data/urheiluruutu/PROG_2020_00824382.mp4_fi.srt
data/urheiluruutu/PROG_2020_00824011.mp4_fi.srt
data/urheiluruutu/PROG_2020_00824065.mp4_fi.srt
data/urheiluruutu/PROG_2020_00823804.mp4_fi.srt
data/urheiluruutu/PROG_2020_00824315.mp4_fi.srt
data/urheiluruutu/PROG_2020_00823937.mp4_fi.srt
data/urheiluruutu/PROG_2020_00823938.mp4_fi.srt
data/urheiluruutu/PROG_2020_00824170.mp4_fi.srt
data/urheiluruutu/PROG_2020_00824275.mp4_fi.srt
data/urheiluruutu/PROG_2020_00823730.mp4_fi.srt
data/urheiluruutu/PROG_2020_00824117.mp4_fi.srt
data/urheiluruutu/PROG_2020_00824381.mp4_fi.srt


In [74]:
df = pd.DataFrame(data)[['program', 'content', 'start', 'end', 'start_s', 'end_s', 'duration_s']]

In [75]:
df.head()

,program,content,start,end,start_s,end_s,duration_s
0,PROG_2020_00824382,-urheilusta,"00:00:01,960","00:00:03,800",1.96,3.80,1.84
1,PROG_2020_00824382,"hyvää iltaa, tuore kysely\npaljastaa hätkähdyt...","00:00:19,480","00:00:23,040",19.48,23.04,3.56
2,PROG_2020_00824382,naisurheilijoiden suhteesta\nomaan kehoonsa mo...,"00:00:23,040","00:00:27,040",23.04,27.04,4.00
3,PROG_2020_00824382,vastanneista määrittelee\noman arvonsa painons...,"00:00:27,040","00:00:31,080",27.04,31.08,4.04
4,PROG_2020_00824382,Ilkka Herola kamppailemaan kun\nmaailmancup ki...,"00:00:31,080","00:00:34,560",31.08,34.56,3.48


In [76]:
! mkdir data/yle_subtitles

mkdir: cannot create directory ‘data/yle_subtitles’: File exists


In [77]:
df.to_csv('data/yle_subtitles/urheiluruutu_subs.csv', index=False)

# Sparql Query for the segments

# Sparql Query for the programs

# Process metadata files

In [133]:
import xml.etree.ElementTree as ET
from tqdm.notebook import tqdm

In [136]:
data_path = '../rdf-converter/data/yle/15-urheiluruutu'
files = os.listdir(data_path)
for ii, file in enumerate(tqdm(files)):

    root = ET.parse(data_path+'/'+file).getroot()
    guid = root.find("./MAObject[1]/GUID").text

    fi_title       = root.find("./MAObject[1]/Meta/[@name='FI_TITLE']").text
    duration_tc    = root.find("./MAObject[1]/Meta/[@name='SYSTEM_DURATION_TC']").text 
    

NameError: name 'encode_uri' is not defined

In [117]:
df_parts.head()

,part,program_uri,program_id,title,start,duration,end
0,http://data.memad.eu/yle/urheiluruutu/4e2526e1...,http://data.memad.eu/yle/urheiluruutu/c5db0f59...,PROG_2020_00823730,Kreikkalais-roomalaisen painin kotimainen kärk...,00:00:14.160,00:02:04.640,00:02:18.800
1,http://data.memad.eu/yle/urheiluruutu/732dab43...,http://data.memad.eu/yle/urheiluruutu/c5db0f59...,PROG_2020_00823730,Jääkiekon SM-liigan peliä (Cmore). Puhelinhaa...,00:02:18.840,00:00:56.920,00:03:15.760
2,http://data.memad.eu/yle/urheiluruutu/483e5e16...,http://data.memad.eu/yle/urheiluruutu/fc98882c...,PROG_2020_00823804,Suomen Cupissa Vantaalla hiihdetään lumipyryss...,00:01:06.560,00:02:05.520,00:03:12.080
3,http://data.memad.eu/yle/urheiluruutu/48c89213...,http://data.memad.eu/yle/urheiluruutu/fc98882c...,PROG_2020_00823804,Kalervo Kummola (SU-) puhelinhaast.,00:00:10.440,00:00:56.080,00:01:06.520
4,http://data.memad.eu/yle/urheiluruutu/0e3855c1...,http://data.memad.eu/yle/urheiluruutu/4234e861...,PROG_2020_00823937,Mäkyppyä: Antti Aallon hypyt Bischofshofenissa...,00:03:25.160,00:02:23.440,00:05:48.600


In [118]:
df_programs = pd.read_csv('data/urheiluruutu_programs.csv')

In [121]:
df_programs.head()

,program_uri,program_id,title,duration,segment_id,program
0,http://data.memad.eu/yle/urheiluruutu/c5db0f59...,PROG_2020_00823730,Urheiluruutu,279.20,PROG_2020_00823730,PROG_2020_00823730
1,http://data.memad.eu/yle/urheiluruutu/fc98882c...,PROG_2020_00823804,Urheiluruutu,279.04,PROG_2020_00823804,PROG_2020_00823804
2,http://data.memad.eu/yle/urheiluruutu/4234e861...,PROG_2020_00823937,Urheiluruutu,608.20,PROG_2020_00823937,PROG_2020_00823937
3,http://data.memad.eu/yle/urheiluruutu/21ad8c88...,PROG_2020_00823938,Urheiluruutu,573.15,PROG_2020_00823938,PROG_2020_00823938
4,http://data.memad.eu/yle/urheiluruutu/2d2f22d1...,PROG_2020_00824011,Urheiluruutu,615.05,PROG_2020_00824011,PROG_2020_00824011


In [122]:
df_programs['segment_id'] = df_programs['program_id']
df_programs['program'] = df_programs['program_id']

df_programs['duration_s'] = df_programs['duration']

In [123]:
df_programs.head()

,program_uri,program_id,title,duration,segment_id,program,duration_s
0,http://data.memad.eu/yle/urheiluruutu/c5db0f59...,PROG_2020_00823730,Urheiluruutu,279.20,PROG_2020_00823730,PROG_2020_00823730,279.20
1,http://data.memad.eu/yle/urheiluruutu/fc98882c...,PROG_2020_00823804,Urheiluruutu,279.04,PROG_2020_00823804,PROG_2020_00823804,279.04
2,http://data.memad.eu/yle/urheiluruutu/4234e861...,PROG_2020_00823937,Urheiluruutu,608.20,PROG_2020_00823937,PROG_2020_00823937,608.20
3,http://data.memad.eu/yle/urheiluruutu/21ad8c88...,PROG_2020_00823938,Urheiluruutu,573.15,PROG_2020_00823938,PROG_2020_00823938,573.15
4,http://data.memad.eu/yle/urheiluruutu/2d2f22d1...,PROG_2020_00824011,Urheiluruutu,615.05,PROG_2020_00824011,PROG_2020_00824011,615.05


In [124]:
df_parts['segment_id'] = df_parts['part'].map(lambda u: u.split('/')[-1])
df_parts['program'] = df_parts['program_id']

df_parts['start_s'] = df_parts['start'].map(to_seconds)
df_parts['end_s'] = df_parts['end'].map(to_seconds)
df_parts['duration_s'] = df_parts['end_s'] - df_parts['start_s']

In [125]:
df_parts.head()

,part,program_uri,program_id,title,start,duration,end,segment_id,program,start_s,end_s,duration_s
0,http://data.memad.eu/yle/urheiluruutu/4e2526e1...,http://data.memad.eu/yle/urheiluruutu/c5db0f59...,PROG_2020_00823730,Kreikkalais-roomalaisen painin kotimainen kärk...,00:00:14.160,00:02:04.640,00:02:18.800,4e2526e14df4c791284d21bd20d0ac61ec02263a,PROG_2020_00823730,14.16,138.80,124.64
1,http://data.memad.eu/yle/urheiluruutu/732dab43...,http://data.memad.eu/yle/urheiluruutu/c5db0f59...,PROG_2020_00823730,Jääkiekon SM-liigan peliä (Cmore). Puhelinhaa...,00:02:18.840,00:00:56.920,00:03:15.760,732dab4376ec366ecd3fd8245c9a84fe7448d162,PROG_2020_00823730,138.84,195.76,56.92
2,http://data.memad.eu/yle/urheiluruutu/483e5e16...,http://data.memad.eu/yle/urheiluruutu/fc98882c...,PROG_2020_00823804,Suomen Cupissa Vantaalla hiihdetään lumipyryss...,00:01:06.560,00:02:05.520,00:03:12.080,483e5e166fd387a117410f4fac887212ad4293c5,PROG_2020_00823804,66.56,192.08,125.52
3,http://data.memad.eu/yle/urheiluruutu/48c89213...,http://data.memad.eu/yle/urheiluruutu/fc98882c...,PROG_2020_00823804,Kalervo Kummola (SU-) puhelinhaast.,00:00:10.440,00:00:56.080,00:01:06.520,48c892139422bc4a573b35ffffd9b38d1dbcffc8,PROG_2020_00823804,10.44,66.52,56.08
4,http://data.memad.eu/yle/urheiluruutu/0e3855c1...,http://data.memad.eu/yle/urheiluruutu/4234e861...,PROG_2020_00823937,Mäkyppyä: Antti Aallon hypyt Bischofshofenissa...,00:03:25.160,00:02:23.440,00:05:48.600,0e3855c1659f1961b1e52b31dac38f245f0c09a9,PROG_2020_00823937,205.16,348.60,143.44


In [126]:
df_segs = df_parts[['segment_id', 'program', 'title', 'start', 'duration', 'start_s', 'duration_s', 'end_s']]

In [127]:
segs = pd.concat([df_segs, df_programs])[['segment_id', 'program', 'title', 'start', 'duration', 'start_s', 'duration_s', 'end_s']]

/home/ismail/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [128]:
segs

,segment_id,program,title,start,duration,start_s,duration_s,end_s
0,4e2526e14df4c791284d21bd20d0ac61ec02263a,PROG_2020_00823730,Kreikkalais-roomalaisen painin kotimainen kärk...,00:00:14.160,00:02:04.640,14.16,124.640,138.800
1,732dab4376ec366ecd3fd8245c9a84fe7448d162,PROG_2020_00823730,Jääkiekon SM-liigan peliä (Cmore). Puhelinhaa...,00:02:18.840,00:00:56.920,138.84,56.920,195.760
2,483e5e166fd387a117410f4fac887212ad4293c5,PROG_2020_00823804,Suomen Cupissa Vantaalla hiihdetään lumipyryss...,00:01:06.560,00:02:05.520,66.56,125.520,192.080
3,48c892139422bc4a573b35ffffd9b38d1dbcffc8,PROG_2020_00823804,Kalervo Kummola (SU-) puhelinhaast.,00:00:10.440,00:00:56.080,10.44,56.080,66.520
4,0e3855c1659f1961b1e52b31dac38f245f0c09a9,PROG_2020_00823937,Mäkyppyä: Antti Aallon hypyt Bischofshofenissa...,00:03:25.160,00:02:23.440,205.16,143.440,348.600
5,6bd4b2d5ba2d7b6e24c4fa62185d77112ab09665,PROG_2020_00823937,Krista Pärmäkoski hiihtää lähdössä ja maastoss...,00:05:48.640,00:02:09.360,348.64,129.360,478.000
6,c2e1c1aa70469241a7287a201c7c1500e7aebeaa,PROG_2020_00823937,Anton Lundell jakaa mitaleita muille joukkueen...,00:00:39.440,00:02:24.320,39.44,144.320,183.760
7,0850e92d52ee1796771ef9c6df15ea362225ab8c,PROG_2020_00823938,Toim Laura Arffman (SU+) selostaa. Sami Jauhoj...,00:07:39.320,00:01:29.520,459.32,89.520,548.840
8,1934354ad2233b7e19c202fb7f02f422aee24b67,PROG_2020_00823938,Heikki Hiltunen (SU-) puhelinhaast.,00:05:27.360,00:00:28.640,327.36,28.640,356.000
9,9b4b3066468437c8d1f1885fdc1e143ab6d52de4,PROG_2020_00823938,Suora: toim Tommi Seppälä (SU+) selostaa. Dall...,00:02:57.760,00:02:29.560,177.76,149.560,327.320


In [129]:
segs.to_csv('data/yle_subtitles/urheiluruutu_segs.csv', index=False)

In [130]:
segs[segs.segment_id.str.startswith('PROG')]

,segment_id,program,title,start,duration,start_s,duration_s,end_s
0,PROG_2020_00823730,PROG_2020_00823730,Urheiluruutu,NaN,279.2,NaN,279.20,NaN
1,PROG_2020_00823804,PROG_2020_00823804,Urheiluruutu,NaN,279.04,NaN,279.04,NaN
2,PROG_2020_00823937,PROG_2020_00823937,Urheiluruutu,NaN,608.2,NaN,608.20,NaN
3,PROG_2020_00823938,PROG_2020_00823938,Urheiluruutu,NaN,573.15,NaN,573.15,NaN
4,PROG_2020_00824011,PROG_2020_00824011,Urheiluruutu,NaN,615.05,NaN,615.05,NaN
5,PROG_2020_00824065,PROG_2020_00824065,Urheiluruutu,NaN,220.09,NaN,220.09,NaN
6,PROG_2020_00824117,PROG_2020_00824117,Urheiluruutu,NaN,592.11,NaN,592.11,NaN
7,PROG_2020_00824170,PROG_2020_00824170,Urheiluruutu,NaN,247.09,NaN,247.09,NaN
8,PROG_2020_00824275,PROG_2020_00824275,Urheiluruutu,NaN,1157.06,NaN,1157.06,NaN
9,PROG_2020_00824315,PROG_2020_00824315,Urheiluruutu,NaN,1169.11,NaN,1169.11,NaN
